In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
import os
# 1. Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이 (숫자)")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")

# 2. 출력 파서
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3. 프롬프트 템플릿
template = """
아래 학생의 자기소개에서 이름, 나이, 전공, 취미 리스트, 목표를 추출해 구조화된 JSON으로 출력해.
반드시 {format_instructions} 형식을 따르세요.

학생 자기소개:
{intro}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["intro"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# 4. LLM 모델
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)
# 5. 체인 실행 함수
def extract_student_info(intro: str):
    input_text = prompt.format(intro=intro)
    output = llm.predict(input_text)
    return parser.parse(output)

# 6. 테스트 입력
intro_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

result = extract_student_info(intro_text)
print(result.dict())
